In [2]:
# Download dataset. This is not needed if you have the dataset already

# import kagglehub

# path = kagglehub.dataset_download("uwrfkaggler/ravdess-emotional-speech-audio")

# print("Path to dataset files:", path)

d:\02Apps\Anaconda\envs\test\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


100%|██████████| 429M/429M [00:52<00:00, 8.59MB/s] 

Extracting files...


Path to dataset files: C:\Users\isaac\.cache\kagglehub\datasets\uwrfkaggler\ravdess-emotional-speech-audio\versions\1


In [2]:
# Function to check the sampling rate of a wav file and valid file path

import wave
import contextlib

def check_sampling_rate(file_path):
    try:
        with contextlib.closing(wave.open(file_path, 'r')) as wav_file:
            sample_rate = wav_file.getframerate()
            print(f"Sampling rate: {sample_rate} Hz")
    except Exception as e:
        print(f"Error: {e}")

# Example usage
file_path = 'data/ravdess-emotional-speech-audio/versions/1/Actor_01/03-01-01-01-01-01-01.wav'
check_sampling_rate(file_path)

Sampling rate: 48000 Hz


In [4]:
"""
File naming convention

Each of the 1440 files has a unique filename. The filename consists of a 7-part numerical identifier (e.g., 03-01-06-01-02-01-12.wav). These identifiers define the stimulus characteristics:

Filename identifiers

Modality (01 = full-AV, 02 = video-only, 03 = audio-only).

Vocal channel (01 = speech, 02 = song).

Emotion (01 = neutral, 02 = calm, 03 = happy, 04 = sad, 05 = angry, 06 = fearful, 07 = disgust, 08 = surprised).

Emotional intensity (01 = normal, 02 = strong). NOTE: There is no strong intensity for the 'neutral' emotion.

Statement (01 = "Kids are talking by the door", 02 = "Dogs are sitting by the door").

Repetition (01 = 1st repetition, 02 = 2nd repetition).
"""


import os
import pandas as pd

emotion_mapping = {
    "01": "neutral",
    "02": "calm",
    "03": "happy",
    "04": "sad",
    "05": "angry",
    "06": "fearful",
    "07": "disgust",
    "08": "surprised"
}

file_dir = "data/ravdess-emotional-speech-audio/versions/1/"

data = []

for actor in os.listdir(file_dir):
    actor_path = os.path.join(file_dir, actor)
    if os.path.isdir(actor_path):
        for file in os.listdir(actor_path):
            if file.endswith(".wav"):
                emotion_code = file[6:8]
                emotion = emotion_mapping.get(emotion_code, "unknown")

                file_path = os.path.join(actor_path, file)

                data.append({"emotion": emotion, "file_path": file_path})

df = pd.DataFrame(data)

print(df)

        emotion                                          file_path
0       neutral  data/ravdess-emotional-speech-audio/versions/1...
1       neutral  data/ravdess-emotional-speech-audio/versions/1...
2       neutral  data/ravdess-emotional-speech-audio/versions/1...
3       neutral  data/ravdess-emotional-speech-audio/versions/1...
4          calm  data/ravdess-emotional-speech-audio/versions/1...
...         ...                                                ...
1435  surprised  data/ravdess-emotional-speech-audio/versions/1...
1436  surprised  data/ravdess-emotional-speech-audio/versions/1...
1437  surprised  data/ravdess-emotional-speech-audio/versions/1...
1438  surprised  data/ravdess-emotional-speech-audio/versions/1...
1439  surprised  data/ravdess-emotional-speech-audio/versions/1...

[1440 rows x 2 columns]
